In [9]:
import pandas as pd
import numpy as np
import sys
from rdkit import Chem, DataStructs
import json
import csv
from rdkit.Chem import AllChem
from tqdm import tqdm 

import os
# sys.path.append(os.path.abspath("/Users/santuchal/Documents/new_ML/final_update_github/new_hERG/hERG/src"))
from new.PubChemFingerprints import *

/var/folders/x6/r97phsxs5qb8n7m2m39tbx0m0000gn/T/ipykernel_71309/2062912925.py:1: DtypeWarning: Columns (118,119,120,124,125,126,130,131,132,136,137,138,142,143,144,148,149,150,154,155,156,160,161,162,166,167,168,172,173,174,268,269,270,274,275,276,280,281,282,286,287,288,292,293,294,298,299,300,304,305,306,310,311,312,316,317,318,322,323,324,327,328,329,332,333,334,337,338,339,342,343,344,347,348,349,352,353,354,357,358,359,362,363,364,367,368,369,372,373,374,377,378,379,382,383,384,387,388,389,392,393,394,397,398,399,402,403,404,407,408,409,412,413,414,417,418,419,422,423,424,480,481,482,483,493,494,495,496,720) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train_validation_cardio_tox_data.csv")


In [8]:
def CalculateECFP2Fingerprint(mol,radius=1): # copied from https://github.com/gadsbyfly/PyBioMed/blob/master/PyBioMed/PyMolecule/fingerprint.py 
	res=AllChem.GetMorganFingerprint(mol,radius)
	fp = tuple(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits = 1024))
	return fp, res.GetNonzeroElements(), res

def calculate_different_fingerprint(smile):
	mol = Chem.MolFromSmiles(smile)
	ECFP2Fingerprint = CalculateECFP2Fingerprint(mol)
	PubChemFingerAll = calcPubChemFingerAll(mol)
	return np.asarray(list(ECFP2Fingerprint[0]) + list(PubChemFingerAll))

file_list = [ "train_validation_cardio_tox_data", "external_test_set_neg", "external_test_set_new", "external_test_set_pos"]

for file_read in file_list:
    df = pd.read_csv("../data/"+str(file_read)+".csv")
    total_activity = df.loc[:,'ACTIVITY']
    total_smiles = df.loc[:,'smiles']
    df = pd.DataFrame()
    i = 0
    total_finger_print = []
    for smile in tqdm(total_smiles):
        total_finger_print.append(calculate_different_fingerprint(smile))
        i = i+1

    with open('../data/MFP_data/'+str(file_read)+'_fingerprint_output.csv', 'w') as writeFile:
        writer = csv.writer(writeFile, delimiter=",")
        writer.writerows(total_finger_print)
    writeFile.close()

/var/folders/x6/r97phsxs5qb8n7m2m39tbx0m0000gn/T/ipykernel_71309/2538337827.py:15: DtypeWarning: Columns (118,119,120,124,125,126,130,131,132,136,137,138,142,143,144,148,149,150,154,155,156,160,161,162,166,167,168,172,173,174,268,269,270,274,275,276,280,281,282,286,287,288,292,293,294,298,299,300,304,305,306,310,311,312,316,317,318,322,323,324,327,328,329,332,333,334,337,338,339,342,343,344,347,348,349,352,353,354,357,358,359,362,363,364,367,368,369,372,373,374,377,378,379,382,383,384,387,388,389,392,393,394,397,398,399,402,403,404,407,408,409,412,413,414,417,418,419,422,423,424,480,481,482,483,493,494,495,496,720) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/"+str(file_read)+".csv")
100%|██████████████████████████████████████████| 44/44 [00:00<00:00, 153.90it/s]


In [11]:
def convert_to_fingerprint_vectorizer(smiles):
    bit_size=1024
    Max_len=93
    dataX = []
    for smile in smiles:
        mol = Chem.MolFromSmiles(smile)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=bit_size)
        fp = np.array(fp)
        dataX.append(fp)
    dataX = np.array(dataX)
    data_x = []
    for i in range(len(dataX)):
        fp = [0] * Max_len
        n_ones = 0
        for j in range(bit_size):
            if dataX[i][j] == 1:
                fp[n_ones] = j+1
                n_ones += 1
        data_x.append(fp)

    data_x = np.array(data_x, dtype=np.int32)
    return data_x


file_list = [ "train_validation_cardio_tox_data", "external_test_set_neg", "external_test_set_new", "external_test_set_pos"]
for file_read in file_list:
    df = pd.read_csv("../data/"+file_read+".csv")
    total_activity = df.iloc[:, 0]#.loc[:,'ACTIVITY']
    total_smiles = df.iloc[:, 1]#.loc[:,'smiles']
    total_fingerprint_vectorizer_data = convert_to_fingerprint_vectorizer(total_smiles)

    with open('../data/FPeV_data/'+file_read+'_vectorizer_data.csv', 'w') as writeFile:
        writer = csv.writer(writeFile, delimiter=",")
        writer.writerows(total_fingerprint_vectorizer_data)
    writeFile.close()


/var/folders/x6/r97phsxs5qb8n7m2m39tbx0m0000gn/T/ipykernel_71309/837221860.py:27: DtypeWarning: Columns (118,119,120,124,125,126,130,131,132,136,137,138,142,143,144,148,149,150,154,155,156,160,161,162,166,167,168,172,173,174,268,269,270,274,275,276,280,281,282,286,287,288,292,293,294,298,299,300,304,305,306,310,311,312,316,317,318,322,323,324,327,328,329,332,333,334,337,338,339,342,343,344,347,348,349,352,353,354,357,358,359,362,363,364,367,368,369,372,373,374,377,378,379,382,383,384,387,388,389,392,393,394,397,398,399,402,403,404,407,408,409,412,413,414,417,418,419,422,423,424,480,481,482,483,493,494,495,496,720) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/"+file_read+".csv")
